# Generators

This notebook was created with information from http://www.bogotobogo.com


By now you should know what iterators are.Generators behave like them but are actually functions and exprssions. The advantage of using a generator is memory efficiency, since instead of loading a whole array into memory it only yields the next value (whenever next() is called)

There are two ways to use generators:

1- As functions: Using yield forces the function to generate one value at a time

2- As expressions: Similar to comprehensions but instead of populating a list it only returns the results on deman in the form of an object

Both use the iteration protocol (which is why we are learning about them in the iterator unit and not in functional programming)

## Yield vs return

Generators are declared just as you would a function, however they have two main differences (or one difference and a complement). Generators do not return a value/object (well that's not entirely true, they might return a StopIteration exception that means that you can't iterate through the generator anymore because it has arrived to the end of it. Whew that's a big parentheses). Instead they __yield__ a new value whenever they are called. For this they heavily rely on the iterator protocol and the \__next__() operator.

Therefore, the very first thing you should learn about generators is how to use the yield statement. Is a special return exclusive to generators (and  merging lanes on a freeway) that tells the Python machine to compute the value on demand. Let's see an example:

In [9]:
def create_counter(n):
	print('create_counter()')
	while True:
		yield n
		print('increment n')
		n += 1

c = create_counter(2)
print(c)
print(next(c))
print(next(c))
print(next(c))

<generator object create_counter at 0x000001C329FA8830>
create_counter()
2
increment n
3
increment n
4


Here's what's going on:

1- We don't need to specify that create_counter is a generator, the presence of yield gives it away. Calling upon it returns a generator that can be used to generate succesive values after n. Note that the first time we call it it yields the original number, that is because we are yielding n before incrementing it.

2- Notice that even though we call creat_counter as a function, it doesn't do anything until we call next() on it. Hence the reason that printing it before calling next just tells us that it is a generator function.

3- The first time we call next(), the generator stops at returning/yielding n. The second time it increments by 1 and then yields it again, and so on.

4- This generator has no stopping condition so it can be called upon infinitely, so be careful on how you use it in a loop.

Let's look at another example using a loop:

In [14]:
def cubic_generator(n):
	for i in range(n):
		yield i ** 3
        
cg=cubic_generator(5)        
for i in cg:
	print(i,  end=' : ')  # Python 3.0
print(next(cg))

0 : 1 : 8 : 27 : 64 : 

StopIteration: 

Notice that this is a finite generator, it has a limit which is determined by the loop inside it. When we try to iterate once more we get a StopIteration exception. If we wanted to use a function instead we would have to generate the whole list of numbers and then iterate through them, or call upon the function everytime we loop (and then we have to create a counter and bla bla bla)

Another example? Sure why not, this one is taken directly from bogotobog so I'll let them explain it to you

[begin citation]

Because generators preserve their local state between invocations, they're particularly well-suited for complicated, stateful iterators, such as fibonacci numbers. The generator returning the Fibonacci numbers using Python's yield statement can be seen below.



In [15]:
def fibonacci(max):
    a, b = 0, 1           # (1)
    while a < max:
        yield a          #  (2)
        a, b = b, a + b   # (3)

for n in fibonacci(500):
	print(n, end=' ')

list(fibonacci(500))

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]

1) It starts with 0 and 1, goes up slowly at first, then more and more rapidly. To start the sequence, we need two variables: a starts at 0, and b starts at 1.

2) a is the current number in the sequence, so yield it.

3) b is the next number in the sequence, so assign that to a, but also calculate the next value a + b and assign that to b for later use. Note that this happens in parallel; if a is 3 and b is 5, then a, b = b, a + b will set a to 5 (the previous value of b) and b to 8 (the sum of the previous values of a and b).

As we can see from the output, we can use a generator like fibonacci() in a for loop directly. The for loop will automatically call the next() function to get values from the fibonacci() generator and assign them to the for loop index variable (n). Each time through the for loop, n gets a new value from the yield statement in fibonacci(), and all we have to do is print it out. Once fibonacci() runs out of numbers (a becomes bigger than max, which in this case is 500), then the for loop exits gracefully.

This is a useful idiom: pass a generator to the list() function, and it will iterate through the entire generator (just like the for loop in the previous example) and return a list of all the values.

[end citation]

## Generators as expressions

Generator expressions are incredibly similar to list comprehensions in the way we call them, the only difference is they use parentheses instead of square brackets.

In [20]:
# List comprehension makes a list
a=[ x ** 3 for x in range(5)]

# Generator expression makes an iterable
b=(x ** 3 for x in range(5))

print(a)
print(b) 
print(next(b))
print(next(b))
print(next(b))

[0, 1, 8, 27, 64]
<generator object <genexpr> at 0x000001C32A094C50>
0
1
8


You can see that the comprehension already generated the list, but the generator does it on demand. Let's look at other examples on how to use a generator as an expression for iterator operators:

In [21]:
sum (x ** 3 for x in range(5))



100

In [22]:
sorted(x ** 3 for x in range(5))



[0, 1, 8, 27, 64]

In [23]:
sorted((x ** 3 for x in range(5)), reverse=True)


[64, 27, 8, 1, 0]

In [24]:
import math
list( map(math.sqrt, (x ** 3 for x in range(5))) )

[0.0, 1.0, 2.8284271247461903, 5.196152422706632, 8.0]

One thing though. Generator expressions are not speed efficient. They are more versatile and more memory efficient than lists, so in general you should only use them for long complicated sequences where memory space is critical.



## Generator.Send()

So far we have seen generators that iterate through predetermined numbers. However, generators can take input as well, this is done through the send() method. Let's look at the following example:

In [40]:
def f():
     while True:
        val = yield
        yield val*10 
g = f()
next(g)
print(g.send(1))
next(g)
print(g.send(10))
next(g)
print(g.send(0.5))

10
100
5.0


We bound the generator f() to g. Then we call upon next so the generator will go directly to yield. Once there we can send a value that is evaluated at yield. Inside the generator we bound that value to val so the generator returns val*10.

In [49]:
import random

def cf():
    while True:
        val = yield
        print (val)

def pf(c):
    while True:
        val = random.randint(1,10)
        c.send(val)
        yield


c = cf()
print(c.send(None))
p = pf(c)

for wow in range(10):
    next(p)

None
9
6
4
10
9
9
8
6
3
9


In this piece of code we are using the cf function to print the value from yield and the pf function to generate a list of random ints. pf send the values to cf which in turn prints it, so we don't need to call on print(next(p)) everytime.

## Final notes on generators

Generators are their own iterator, therefore there can only be one active iteration. You cannot retrieve values from two different passes at a generator, and once its exhausted you need to start a new one. If you need to keep the values of a generator for future use you need to bind them to a list, tuple or dictionary. here's an example:

In [25]:
G = (c * 5 for c in 'Python')
 # Iterate manually
I1 = iter(G)
print(next(I1))

print( next(I1))

I2 = iter(G)
print( next(I2))


PPPPP
yyyyy
ttttt


Even though the generator is bound to two different objects, it points to the same spot unless a new generator is created.

Finally, you can get away without knowing generators. Lists, tuples and dictionaries will get you by in creating solutions to most of the coding problems you will encounter. However, knowing them and using them, just as using lambdas and comprehensions, are the true test of a master Pythonista. Whenever someone asks you if you know Python more often than not the next wquestion will be do you know generators, lambdas and comprehensions? to which I hope you truthfully answer with "('yes'for x in range(3))"

In [32]:
affirmative=('yes'for x in range(3))
print(next(affirmative))
print(next(affirmative))
print(next(affirmative))

yes
yes
yes
